In [102]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

In [103]:
#read the data
#laptop
#df_DropSeq_MCF7 = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\MCF7_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
#desktop
#C:\Users\Cameron\Desktop\AILab24\DropSeq
MCF = pd.read_csv("C:\\Users\\Cameron\\Desktop\\AILab24\\DropSeq\\MCF7_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)

In [104]:
#add a new row to the data frame
#add the row to the bottom
#if at the column index contains _Normoxia then add 1 to the row, _Hypoxia then add -1
#this is just binary classification

In [105]:
#function to add new 'Oxia' row to the data frame
#1 is Normoxia, 0 is Hypoxia
def oxia(df):
    new_row = {col: None for col in df.columns}
    for col in df.columns:
        if '_Normoxia' in col:
            new_row[col] = 1
        elif '_Hypoxia' in col:
            new_row[col] = 0
    df = pd.concat([df, pd.DataFrame(new_row, index=['oxia'])])
    return df

In [106]:
MCF = oxia(MCF)
MCF

,"""AAAAACCTATCG_Normoxia""","""AAAACAACCCTA_Normoxia""","""AAAACACTCTCA_Normoxia""","""AAAACCAGGCAC_Normoxia""","""AAAACCTAGCTC_Normoxia""","""AAAACCTCCGGG_Normoxia""","""AAAACTCGTTGC_Normoxia""","""AAAAGAGCTCTC_Normoxia""","""AAAAGCTAGGCG_Normoxia""","""AAAATCGCATTT_Normoxia""",...,"""TTTTACAGGATC_Hypoxia""","""TTTTACCACGTA_Hypoxia""","""TTTTATGCTACG_Hypoxia""","""TTTTCCAGACGC_Hypoxia""","""TTTTCGCGCTCG_Hypoxia""","""TTTTCGCGTAGA_Hypoxia""","""TTTTCGTCCGCT_Hypoxia""","""TTTTCTCCGGCT_Hypoxia""","""TTTTGTTCAAAG_Hypoxia""","""TTTTTTGTATGT_Hypoxia"""
"""MALAT1""",1,3,3,6,4,5,1,13,3,3,...,0,2,1,0,1,0,1,0,0,4
"""MT-RNR2""",0,0,0,2,0,0,2,1,7,0,...,0,0,0,0,0,0,0,0,0,0
"""NEAT1""",0,0,0,0,0,2,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
"""H1-5""",0,0,0,0,0,2,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
"""TFF1""",4,1,1,1,0,0,0,2,0,1,...,2,3,8,0,0,3,4,2,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""RPS19BP1""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AUNIP""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""TNK2""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""SUDS3""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [107]:
#now to the data
#transpose the data
MCF = MCF.T
MCF

,"""MALAT1""","""MT-RNR2""","""NEAT1""","""H1-5""","""TFF1""","""MT-RNR1""","""H4C3""","""GDF15""","""KRT81""","""MT-CO3""",...,"""SKIDA1""","""MICALL1""","""RARG""","""MYO1F""","""BRWD1-AS2""","""RPS19BP1""","""AUNIP""","""TNK2""","""SUDS3""",oxia
"""AAAAACCTATCG_Normoxia""",1,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAACAACCCTA_Normoxia""",3,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAACACTCTCA_Normoxia""",3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAACCAGGCAC_Normoxia""",6,2,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAACCTAGCTC_Normoxia""",4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""TTTTCGCGTAGA_Hypoxia""",0,0,0,0,3,0,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCGTCCGCT_Hypoxia""",1,0,0,0,4,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCTCCGGCT_Hypoxia""",0,0,0,1,2,0,4,0,0,0,...,0,0,0,0,0,0,0,0,1,0
"""TTTTGTTCAAAG_Hypoxia""",0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
#count the number of hypoxic and normoxic samples
MCF['oxia'].value_counts()

oxia
1    12705
0     8921
Name: count, dtype: int64

In [109]:
#now we need to split the data into training and testing data
#remove the oxia row and store it in a variable
y_MCF = MCF['oxia']
X_MCF = MCF.drop('oxia', axis=1)

In [110]:
X_MCF #this is all the data, needs to be split for testing and training

,"""MALAT1""","""MT-RNR2""","""NEAT1""","""H1-5""","""TFF1""","""MT-RNR1""","""H4C3""","""GDF15""","""KRT81""","""MT-CO3""",...,"""MROH1""","""SKIDA1""","""MICALL1""","""RARG""","""MYO1F""","""BRWD1-AS2""","""RPS19BP1""","""AUNIP""","""TNK2""","""SUDS3"""
"""AAAAACCTATCG_Normoxia""",1,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACAACCCTA_Normoxia""",3,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACACTCTCA_Normoxia""",3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACCAGGCAC_Normoxia""",6,2,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACCTAGCTC_Normoxia""",4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""TTTTCGCGTAGA_Hypoxia""",0,0,0,0,3,0,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCGTCCGCT_Hypoxia""",1,0,0,0,4,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCTCCGGCT_Hypoxia""",0,0,0,1,2,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""TTTTGTTCAAAG_Hypoxia""",0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
y_MCF #this is the target data, were trying to predict this

"AAAAACCTATCG_Normoxia"    1
"AAAACAACCCTA_Normoxia"    1
"AAAACACTCTCA_Normoxia"    1
"AAAACCAGGCAC_Normoxia"    1
"AAAACCTAGCTC_Normoxia"    1
                          ..
"TTTTCGCGTAGA_Hypoxia"     0
"TTTTCGTCCGCT_Hypoxia"     0
"TTTTCTCCGGCT_Hypoxia"     0
"TTTTGTTCAAAG_Hypoxia"     0
"TTTTTTGTATGT_Hypoxia"     0
Name: oxia, Length: 21626, dtype: int64

In [112]:
#split the data into training and testing data
X_train_MCF, X_test_MCF, y_train_MCF, y_test_MCF = train_test_split(X_MCF, y_MCF, test_size=0.2, random_state=42)
#lets see how they are split

In [113]:
#and their shapes:
print("X_train shape: ", X_train_MCF.shape, "X_test shape: ", X_test_MCF.shape, "y_train shape: ", y_train_MCF.shape, "y_test shape: ", y_test_MCF.shape)

X_train shape:  (17300, 3000) X_test shape:  (4326, 3000) y_train shape:  (17300,) y_test shape:  (4326,)


In [114]:
#now to train the model
#lets start with simple linear regression and see how it goes
# Step 1: Initialize the Linear Regression model
lin_reg_MCF = LinearRegression()
# Step 2: Fit the model on the training data
lin_reg_MCF.fit(X_train_MCF, y_train_MCF)

# Step 3: Predict the target variable for the testing data
y_pred_MCF = lin_reg_MCF.predict(X_test_MCF)

# Step 4: Evaluate the model's performance
mse = mean_squared_error(y_test_MCF, y_pred_MCF)
r2 = r2_score(y_test_MCF, y_pred_MCF)
y_pred_MCF = np.round(y_pred_MCF)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")
print("Confusion Matrix: ", confusion_matrix(y_test_MCF, y_pred_MCF))
print("Accuracy: ", accuracy_score(y_test_MCF, y_pred_MCF))

Mean Squared Error: 0.06182092653045139
R^2 Score: 0.7438691949867688
Confusion Matrix:  [[   0    0    0    0]
 [  23 1639   99    0]
 [   0   51 2501   13]
 [   0    0    0    0]]
Accuracy:  0.957004160887656


In [115]:
#because we already have binary output data it might be worth to try a logistic regression model
logistic_MCF7 = LogisticRegression(solver='liblinear')
params = {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
logistic_gs_MCF7 = GridSearchCV(logistic_MCF7, params, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
logistic_gs_MCF7.fit(X_train_MCF, y_train_MCF)

GridSearchCV(cv=5, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'penalty': ['l1', 'l2']},
             refit='neg_log_loss', scoring=['neg_log_loss'])

In [116]:
best_log_MCF = logistic_gs_MCF7.best_estimator_
print(best_log_MCF)
print(f"The best parameters for the logistic regression:{logistic_gs_MCF7.best_params_}")

LogisticRegression(C=0.1, solver='liblinear')
The best parameters for the logistic regression:{'C': 0.1, 'penalty': 'l2'}


In [117]:
#How accurate were the predictions?
y_pred_MCF = best_log_MCF.predict(X_test_MCF)
accuracy_logistic = accuracy_score(y_test_MCF, y_pred_MCF)
print(f"Accuracy: {accuracy_logistic}")
print("Confusion Matrix: ", confusion_matrix(y_test_MCF, y_pred_MCF))
print("Mean Squared Error: ", mean_squared_error(y_test_MCF, y_pred_MCF))
print("R^2 Score: ", r2_score(y_test_MCF, y_pred_MCF))

Accuracy: 0.9787332408691632
Confusion Matrix:  [[1708   53]
 [  39 2526]]
Mean Squared Error:  0.0212667591308368
R^2 Score:  0.9118895098810816


In [118]:
#k nearest neighbours
knn_MCF = KNeighborsClassifier()
params = {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance']}
knn_gs_MCF = GridSearchCV(knn_MCF, params, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
knn_gs_MCF.fit(X_train_MCF, y_train_MCF)
best_knn_MCF = knn_gs_MCF.best_estimator_

In [119]:
best_knn_MCF.fit(X_train_MCF, y_train_MCF)
y_pred_MCF = best_knn_MCF.predict(X_test_MCF)

In [120]:
print(f"Accuracy: {accuracy_score(y_test_MCF, y_pred_MCF)}")
print("Confusion Matrix: ", confusion_matrix(y_test_MCF, y_pred_MCF))
print("Mean Squared Error: ", mean_squared_error(y_test_MCF, y_pred_MCF))
print("R^2 Score: ", r2_score(y_test_MCF, y_pred_MCF))

Accuracy: 0.889967637540453
Confusion Matrix:  [[1706   55]
 [ 421 2144]]
Mean Squared Error:  0.11003236245954692
R^2 Score:  0.5441239859064659


In [121]:
#random forest
forest_MCF = RandomForestClassifier(random_state=42)
params = {'n_estimators': [10, 50, 100, 200], 'max_depth': [10, 50, 100, 200]}
forest_gs_MCF = GridSearchCV(forest_MCF, params, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
forest_gs_MCF.fit(X_train_MCF, y_train_MCF)
best_forest_MCF = forest_gs_MCF.best_estimator_

In [122]:
best_forest_MCF.fit(X_train_MCF, y_train_MCF)
y_pred_MCF = best_forest_MCF.predict(X_test_MCF)

In [123]:
print(f"Accuracy: {accuracy_score(y_test_MCF, y_pred_MCF)}")
print("Confusion Matrix: ", confusion_matrix(y_test_MCF, y_pred_MCF))
print("Mean Squared Error: ", mean_squared_error(y_test_MCF, y_pred_MCF))
print("R^2 Score: ", r2_score(y_test_MCF, y_pred_MCF))

Accuracy: 0.9678687008784096
Confusion Matrix:  [[1639  122]
 [  17 2548]]
Mean Squared Error:  0.032131299121590386
R^2 Score:  0.8668765421029386


In [124]:
#now to try a different model
#how about perceptron multilayer perceptron

# Step 1: Initialize the MLP Classifier model
NN_MLP_MCF = MLPClassifier(random_state=42, batch_size='auto', max_iter=1000000, solver='sgd')
# Step 2: Fit the model on the training data
params = {'hidden_layer_sizes': [(50,), (100,), (50,50), (100,100)], 'learning_rate_init': [0.1, 0.01, 0.001]}
NN_MLP_gs_MCF = GridSearchCV(NN_MLP_MCF, params, cv=3, verbose=2)
NN_MLP_gs_MCF.fit(X_train_MCF, y_train_MCF)
NN_best_MCF = NN_MLP_gs_MCF.best_estimator_
NN_best_MCF.fit(X_train_MCF, y_train_MCF)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END ...hidden_layer_sizes=(50,), learning_rate_init=0.1; total time=   8.2s
[CV] END ...hidden_layer_sizes=(50,), learning_rate_init=0.1; total time=  10.7s
[CV] END ...hidden_layer_sizes=(50,), learning_rate_init=0.1; total time=  12.6s
[CV] END ..hidden_layer_sizes=(50,), learning_rate_init=0.01; total time=  26.4s
[CV] END ..hidden_layer_sizes=(50,), learning_rate_init=0.01; total time=  26.6s
[CV] END ..hidden_layer_sizes=(50,), learning_rate_init=0.01; total time=  27.3s
[CV] END .hidden_layer_sizes=(50,), learning_rate_init=0.001; total time= 1.7min
[CV] END .hidden_layer_sizes=(50,), learning_rate_init=0.001; total time= 1.8min
[CV] END .hidden_layer_sizes=(50,), learning_rate_init=0.001; total time= 1.5min
[CV] END ..hidden_layer_sizes=(100,), learning_rate_init=0.1; total time=  19.8s
[CV] END ..hidden_layer_sizes=(100,), learning_rate_init=0.1; total time=  16.3s
[CV] END ..hidden_layer_sizes=(100,), learning_r

MLPClassifier(hidden_layer_sizes=(100, 100), learning_rate_init=0.01,
              max_iter=1000000, random_state=42, solver='sgd')

In [125]:
# Step 3: Predict the target variable for the testing data
y_pred_MCF = NN_best_MCF.predict(X_test_MCF)
#step 4: Evaluate the model's performance
print(f"Accuracy: {accuracy_score(y_test_MCF, y_pred_MCF)}")
print("Confusion Matrix: ", confusion_matrix(y_test_MCF, y_pred_MCF))
print("Mean Squared Error: ", mean_squared_error(y_test_MCF, y_pred_MCF))
print("R^2 Score: ", r2_score(y_test_MCF, y_pred_MCF))

Accuracy: 0.9752658344891355
Confusion Matrix:  [[1677   84]
 [  23 2542]]
Mean Squared Error:  0.02473416551086454
R^2 Score:  0.8975236691008233


In [126]:
#ensembling
#create a dictionary of our models
estimators_MCF = [('logistic', best_log_MCF), ('forest', best_forest_MCF), ('knn', best_knn_MCF)]
#create our voting classifier, inputting our models
ensemble_MCF = VotingClassifier(estimators_MCF, voting='soft')
#fit model to training data
ensemble_MCF.fit(X_train_MCF, y_train_MCF)

VotingClassifier(estimators=[('logistic',
                              LogisticRegression(C=0.1, solver='liblinear')),
                             ('forest',
                              RandomForestClassifier(max_depth=100,
                                                     n_estimators=200,
                                                     random_state=42)),
                             ('knn',
                              KNeighborsClassifier(n_neighbors=11,
                                                   weights='distance'))],
                 voting='soft')

In [127]:
#predict the target variable for the testing data
y_pred_MCF = ensemble_MCF.predict(X_test_MCF)
#evaluate the model's performance
print(f"Accuracy: {accuracy_score(y_test_MCF, y_pred_MCF)}")
print("Confusion Matrix: ", confusion_matrix(y_test_MCF, y_pred_MCF))
print("Mean Squared Error: ", mean_squared_error(y_test_MCF, y_pred_MCF))
print("R^2 Score: ", r2_score(y_test_MCF, y_pred_MCF))

Accuracy: 0.9801202034211743
Confusion Matrix:  [[1711   50]
 [  36 2529]]
Mean Squared Error:  0.019879796578825704
R^2 Score:  0.9176358461931851


In [128]:
#what if we try with the hcc data
#laptop
#df = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\HCC1806_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
#desktop
#C:\Users\Cameron\Desktop\AILab24\DropSeq
HCC = pd.read_csv("C:\\Users\\Cameron\\Desktop\\AILab24\\DropSeq\\HCC1806_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
HCC

,"""AAAAAACCCGGC_Normoxia""","""AAAACCGGATGC_Normoxia""","""AAAACGAGCTAG_Normoxia""","""AAAACTTCCCCG_Normoxia""","""AAAAGCCTACCC_Normoxia""","""AAACACAAATCT_Normoxia""","""AAACCAAGCCCA_Normoxia""","""AAACCATGCACT_Normoxia""","""AAACCTCCGGCT_Normoxia""","""AAACGCCGGTCC_Normoxia""",...,"""TTTTCTGATGGT_Hypoxia""","""TTTTGATTCAGA_Hypoxia""","""TTTTGCAACTGA_Hypoxia""","""TTTTGCCGGGCC_Hypoxia""","""TTTTGTTAGCCT_Hypoxia""","""TTTTTACCAATC_Hypoxia""","""TTTTTCCGTGCA_Hypoxia""","""TTTTTGCCTGGG_Hypoxia""","""TTTTTGTAACAG_Hypoxia""","""TTTTTTTGAATC_Hypoxia"""
"""H1-5""",2,2,5,1,0,0,0,0,1,0,...,0,1,0,2,1,0,0,0,3,1
"""MALAT1""",3,3,2,3,12,3,1,2,0,0,...,3,1,1,1,4,0,4,1,3,6
"""MT-RNR2""",0,0,0,0,0,0,0,0,0,1,...,1,2,2,2,0,0,1,0,1,0
"""ARVCF""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""BCYRN1""",0,1,1,0,0,1,1,2,0,3,...,1,1,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""SCCPDH""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""NTAN1""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
"""CLIP2""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""DUSP23""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [129]:
#same process as before
HCC = oxia(HCC)
HCC

,"""AAAAAACCCGGC_Normoxia""","""AAAACCGGATGC_Normoxia""","""AAAACGAGCTAG_Normoxia""","""AAAACTTCCCCG_Normoxia""","""AAAAGCCTACCC_Normoxia""","""AAACACAAATCT_Normoxia""","""AAACCAAGCCCA_Normoxia""","""AAACCATGCACT_Normoxia""","""AAACCTCCGGCT_Normoxia""","""AAACGCCGGTCC_Normoxia""",...,"""TTTTCTGATGGT_Hypoxia""","""TTTTGATTCAGA_Hypoxia""","""TTTTGCAACTGA_Hypoxia""","""TTTTGCCGGGCC_Hypoxia""","""TTTTGTTAGCCT_Hypoxia""","""TTTTTACCAATC_Hypoxia""","""TTTTTCCGTGCA_Hypoxia""","""TTTTTGCCTGGG_Hypoxia""","""TTTTTGTAACAG_Hypoxia""","""TTTTTTTGAATC_Hypoxia"""
"""H1-5""",2,2,5,1,0,0,0,0,1,0,...,0,1,0,2,1,0,0,0,3,1
"""MALAT1""",3,3,2,3,12,3,1,2,0,0,...,3,1,1,1,4,0,4,1,3,6
"""MT-RNR2""",0,0,0,0,0,0,0,0,0,1,...,1,2,2,2,0,0,1,0,1,0
"""ARVCF""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""BCYRN1""",0,1,1,0,0,1,1,2,0,3,...,1,1,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""NTAN1""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
"""CLIP2""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""DUSP23""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""ZNF682""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
HCC = HCC.T
HCC

,"""H1-5""","""MALAT1""","""MT-RNR2""","""ARVCF""","""BCYRN1""","""ATXN7L2""","""IGFBP3""","""H1-3""","""CTIF""","""RNF123""",...,"""CDKN3""","""DLD""","""PMPCA""","""ZNF165""","""SCCPDH""","""NTAN1""","""CLIP2""","""DUSP23""","""ZNF682""",oxia
"""AAAAAACCCGGC_Normoxia""",2,3,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAACCGGATGC_Normoxia""",2,3,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAACGAGCTAG_Normoxia""",5,2,0,0,1,0,1,3,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAACTTCCCCG_Normoxia""",1,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAAGCCTACCC_Normoxia""",0,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""TTTTTACCAATC_Hypoxia""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTTCCGTGCA_Hypoxia""",0,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTTGCCTGGG_Hypoxia""",0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTTGTAACAG_Hypoxia""",3,3,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
HCC['oxia'].value_counts()

oxia
0    8899
1    5783
Name: count, dtype: int64

In [132]:
y_HCC = HCC['oxia']
X_HCC = HCC.drop('oxia', axis=1)

In [133]:
X_train_HCC, X_test_HCC, y_train_HCC, y_test_HCC = train_test_split(X_HCC, y_HCC, test_size=0.2, random_state=42)

In [134]:
#try the same methods as before, first a linear regression, then try a logistic regression
lin_reg_HCC = LinearRegression()
lin_reg_HCC.fit(X_train_HCC, y_train_HCC)
y_pred_HCC = lin_reg_HCC.predict(X_test_HCC)
mse = mean_squared_error(y_test_HCC, y_pred_HCC)
r2 = r2_score(y_test_HCC, y_pred_HCC)
y_pred_HCC = np.round(y_pred_HCC)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")
print("Confusion Matrix: ", confusion_matrix(y_test_HCC, y_pred_HCC))
print("Accuracy: ", accuracy_score(y_test_HCC, y_pred_HCC))

Mean Squared Error: 0.09161015251015706
R^2 Score: 0.6178763610402165
Confusion Matrix:  [[   0    0    0    0]
 [  34 1599  132    1]
 [   0   90 1078    3]
 [   0    0    0    0]]
Accuracy:  0.9114742934967655


In [135]:
logistic_HCC = LogisticRegression(solver='liblinear')
params = {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
logistic_gs_HCC = GridSearchCV(logistic_HCC, params, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
logistic_gs_HCC.fit(X_train_HCC, y_train_HCC)
best_log_HCC = logistic_gs_HCC.best_estimator_
print(best_log_HCC)
print(f"The best parameters for the logistic regression:{logistic_gs_HCC.best_params_}")

LogisticRegression(C=0.1, solver='liblinear')
The best parameters for the logistic regression:{'C': 0.1, 'penalty': 'l2'}


In [136]:
y_pred_HCC = best_log_HCC.predict(X_test_HCC)
accuracy = accuracy_score(y_test_HCC, y_pred_HCC)
print(f"Accuracy: {accuracy_logistic}")
print("Confusion Matrix: ", confusion_matrix(y_test_HCC, y_pred_HCC))
print("Mean Squared Error: ", mean_squared_error(y_test_HCC, y_pred_HCC))
print("R^2 Score: ", r2_score(y_test_HCC, y_pred_HCC))

Accuracy: 0.9787332408691632
Confusion Matrix:  [[1684   82]
 [  50 1121]]
Mean Squared Error:  0.0449438202247191
R^2 Score:  0.8125306457587237


In [137]:
#k nearest neighbors
knn_HCC = KNeighborsClassifier()
params = {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance']}
knn_gs_HCC = GridSearchCV(knn_HCC, params, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
knn_gs_HCC.fit(X_train_HCC, y_train_HCC)
best_knn_HCC = knn_gs_HCC.best_estimator_

In [138]:
best_knn_HCC.fit(X_train_HCC, y_train_HCC)
y_pred_HCC = best_knn_HCC.predict(X_test_HCC)

In [139]:
print(f"Accuracy: {accuracy_score(y_test_HCC, y_pred_HCC)}")
print("Confusion Matrix: ", confusion_matrix(y_test_HCC, y_pred_HCC))
print("Mean Squared Error: ", mean_squared_error(y_test_HCC, y_pred_HCC))
print("R^2 Score: ", r2_score(y_test_HCC, y_pred_HCC))

Accuracy: 0.5655430711610487
Confusion Matrix:  [[ 496 1270]
 [   6 1165]]
Mean Squared Error:  0.4344569288389513
R^2 Score:  -0.8122037576656709


In [140]:
#random forest
forest_HCC = RandomForestClassifier(random_state=42)
params = {'n_estimators': [10, 50, 100, 200], 'max_depth': [10, 50, 100, 200]}
forest_gs_HCC = GridSearchCV(forest_HCC, params, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
forest_gs_HCC.fit(X_train_HCC, y_train_HCC)
best_forest_HCC = forest_gs_HCC.best_estimator_

In [141]:
best_forest_HCC.fit(X_train_HCC, y_train_HCC)
y_pred_HCC = best_forest_HCC.predict(X_test_HCC)

In [142]:
print(f"Accuracy: {accuracy_score(y_test_HCC, y_pred_HCC)}")
print("Confusion Matrix: ", confusion_matrix(y_test_HCC, y_pred_HCC))
print("Mean Squared Error: ", mean_squared_error(y_test_HCC, y_pred_HCC))
print("R^2 Score: ", r2_score(y_test_HCC, y_pred_HCC))

Accuracy: 0.9244126659856997
Confusion Matrix:  [[1726   40]
 [ 182  989]]
Mean Squared Error:  0.0755873340143003
R^2 Score:  0.684710631503308


In [143]:
#time for a neural network
NN_MLP_HCC = MLPClassifier(random_state=42, batch_size='auto', max_iter=1000000, solver='sgd')
params = {'hidden_layer_sizes': [(50,), (100,), (50,50), (100,100)], 'learning_rate_init': [0.1, 0.01, 0.001]}
NN_MLP_gs_HCC = GridSearchCV(NN_MLP_HCC, params, cv=3, verbose=2)
NN_MLP_gs_HCC.fit(X_train_HCC, y_train_HCC)
NN_best_HCC = NN_MLP_gs_HCC.best_estimator_

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END ...hidden_layer_sizes=(50,), learning_rate_init=0.1; total time=   5.5s
[CV] END ...hidden_layer_sizes=(50,), learning_rate_init=0.1; total time=   5.6s
[CV] END ...hidden_layer_sizes=(50,), learning_rate_init=0.1; total time=   5.4s
[CV] END ..hidden_layer_sizes=(50,), learning_rate_init=0.01; total time=  19.3s
[CV] END ..hidden_layer_sizes=(50,), learning_rate_init=0.01; total time=  18.3s
[CV] END ..hidden_layer_sizes=(50,), learning_rate_init=0.01; total time=  18.4s
[CV] END .hidden_layer_sizes=(50,), learning_rate_init=0.001; total time= 1.3min
[CV] END .hidden_layer_sizes=(50,), learning_rate_init=0.001; total time= 1.2min
[CV] END .hidden_layer_sizes=(50,), learning_rate_init=0.001; total time= 1.2min
[CV] END ..hidden_layer_sizes=(100,), learning_rate_init=0.1; total time=   8.1s
[CV] END ..hidden_layer_sizes=(100,), learning_rate_init=0.1; total time=   8.0s
[CV] END ..hidden_layer_sizes=(100,), learning_r

In [144]:
NN_best_HCC.fit(X_train_HCC, y_train_HCC)
y_pred_HCC = NN_best_HCC.predict(X_test_HCC)

In [145]:
print(f"Accuracy: {accuracy_score(y_test_HCC, y_pred_HCC)}")
print("Confusion Matrix: ", confusion_matrix(y_test_HCC, y_pred_HCC))
print("Mean Squared Error: ", mean_squared_error(y_test_HCC, y_pred_HCC))
print("R^2 Score: ", r2_score(y_test_HCC, y_pred_HCC))

Accuracy: 0.9570990806945863
Confusion Matrix:  [[1678   88]
 [  38 1133]]
Mean Squared Error:  0.04290091930541369
R^2 Score:  0.8210519800424181


In [146]:
#ensembling
estimators_HCC = [('logistic', best_log_HCC), ('forest', best_forest_HCC), ('knn', best_knn_HCC)]
ensemble_HCC = VotingClassifier(estimators_HCC, voting='soft')
ensemble_HCC.fit(X_train_HCC, y_train_HCC)

VotingClassifier(estimators=[('logistic',
                              LogisticRegression(C=0.1, solver='liblinear')),
                             ('forest',
                              RandomForestClassifier(max_depth=100,
                                                     random_state=42)),
                             ('knn', KNeighborsClassifier(n_neighbors=11))],
                 voting='soft')

In [147]:
y_pred = ensemble_HCC.predict(X_test_HCC)
print(f"Accuracy: {accuracy_score(y_test_HCC, y_pred_HCC)}")
print("Confusion Matrix: ", confusion_matrix(y_test_HCC, y_pred_HCC))
print("Mean Squared Error: ", mean_squared_error(y_test_HCC, y_pred_HCC))
print("R^2 Score: ", r2_score(y_test_HCC, y_pred_HCC))

Accuracy: 0.9570990806945863
Confusion Matrix:  [[1678   88]
 [  38 1133]]
Mean Squared Error:  0.04290091930541369
R^2 Score:  0.8210519800424181
